### GOAL - Query big data using AI Agents without writing big data analytics code. <br>
The AI Agent will use built-in strands tools and MCP server to get the job done

## Environment Setup
Install required dependencies for the notebook including Strands SDK, AWS SDK, and MCP client libraries.

In [2]:
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%m/%d %H:%M:%S',
    filename='strands_debug.log'
)


### Pre-Requisites
You have run the Notebook 1

### Import Dependencies and AWS Configuration
Import required libraries and configure AWS settings for the data processing workflow.

In [3]:
# Import required libraries
import os, time, boto3, json
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import use_aws, file_write, file_read, file_write, sleep, python_repl
from datetime import datetime
from pprint import pprint
from pydantic import BaseModel, Field
from typing import Optional, List

# Bypass tool consent for automated execution
os.environ["BYPASS_TOOL_CONSENT"] = "true"
# Specify that if python_repl tool is used, it shouldnt wait for user interaction
os.environ["PYTHON_REPL_INTERACTIVE"] = "false"

## Get Metadata Info from the json file that we created in the previous notebook


In [6]:
# Read the file metadata.json into a dictionary without using file_read
with open('metadata.json', 'r') as f:
    db_metadata = json.load(f)
db_metadata

{'database': {'name': 'awslabbigdataagentstack_db',
  'purpose': 'Stores NYC taxi trip data for analysis and reporting purposes'},
 'tables': [{'name': 'data',
   'purpose': 'Contains detailed NYC taxi trip records including pickup/dropoff times, locations, fares, and other trip-related information',
   'storage_location': 's3://000384376621-us-west-2-big-data-agent-workshop/data/',
   'format': 'parquet',
   'record_count': 9019594,
   'partitioning': [{'name': 'taxi_class',
     'type': 'string',
     'purpose': 'Partitions data by taxi type (yellow or green)'},
    {'name': 'year',
     'type': 'string',
     'purpose': 'Partitions data by year of trip'},
    {'name': 'month',
     'type': 'string',
     'purpose': 'Partitions data by month of trip'}],
   'columns': [{'name': 'vendorid',
     'type': 'int',
     'purpose': 'Identifier for the taxi vendor/provider company'},
    {'name': 'lpep_pickup_datetime',
     'type': 'timestamp',
     'purpose': 'Date and time when the passeng

## MCP Client Setup
We wll get tools exposed by an MCP server to discover partition columns and keys from the data in S3.<br><br>
Initialize the AWS Data Processing MCP server client to provide AI agents with AWS Glue, EMR, and Athena capabilities.

In [7]:
# Import MCP client libraries
from mcp import stdio_client, StdioServerParameters
from strands.tools.mcp import MCPClient

# Create MCP client for AWS data processing server
# This provides tools for Glue, EMR, and Athena operations
import boto3
session = boto3.Session()
credentials = session.get_credentials()

# Create MCP client for AWS data processing server
# This provides tools for Glue, EMR, and Athena operations
data_mcp_client = MCPClient(lambda: stdio_client(
    StdioServerParameters(
        command="uvx",  # Use uvx to run the MCP server
        args= [
            "awslabs.aws-dataprocessing-mcp-server@latest",
            "--allow-write",  # Enable write operations
        ],
        env= {
            "AWS_ACCESS_KEY_ID": credentials.access_key,
            "AWS_SECRET_ACCESS_KEY": credentials.secret_key,
            "AWS_SESSION_TOKEN": credentials.token,
            "FASTMCP_LOG_LEVEL": "ERROR",  # Minimize logging noise
            "AWS_REGION": session.region_name      # Set AWS region
      }
    )
))

### Let's Ask Natural Language Questions to AI Agent

In [10]:
from utils_big_data import print_tokens_costs, load_system_prompt_from_file
from IPython.display import display, Markdown

# Let's load the system prompt from file for running queries on data in S3 data lake
query_system_prompt = load_system_prompt_from_file("text_to_sql_prompt.txt", db_metadata=db_metadata)
display(Markdown(query_system_prompt))

You are an expert AWS data analyst assistant specializing in querying data stored in S3 data lakes using AWS Glue and Amazon Athena and generating responses in natural language so humans can understand.

## Core Responsibilities
- Translate natural language questions into efficient, optimized Amazon Athena-compatible SQL queries
- Execute queries using Amazon Athena
- Respond back to the query in natural language

## Technical Configuration
- Use the database, table, and column definition from here: 
{'database': {'name': 'awslabbigdataagentstack_db', 'purpose': 'Stores NYC taxi trip data for analysis and reporting purposes'}, 'tables': [{'name': 'data', 'purpose': 'Contains detailed NYC taxi trip records including pickup/dropoff times, locations, fares, and other trip-related information', 'storage_location': 's3://000384376621-us-west-2-big-data-agent-workshop/data/', 'format': 'parquet', 'record_count': 9019594, 'partitioning': [{'name': 'taxi_class', 'type': 'string', 'purpose': 'Partitions data by taxi type (yellow or green)'}, {'name': 'year', 'type': 'string', 'purpose': 'Partitions data by year of trip'}, {'name': 'month', 'type': 'string', 'purpose': 'Partitions data by month of trip'}], 'columns': [{'name': 'vendorid', 'type': 'int', 'purpose': 'Identifier for the taxi vendor/provider company'}, {'name': 'lpep_pickup_datetime', 'type': 'timestamp', 'purpose': 'Date and time when the passenger was picked up (used for green taxis)'}, {'name': 'lpep_dropoff_datetime', 'type': 'timestamp', 'purpose': 'Date and time when the passenger was dropped off (used for green taxis)'}, {'name': 'store_and_fwd_flag', 'type': 'string', 'purpose': "Flag indicating whether the trip record was stored in vehicle memory before sending to the vendor ('Y' for yes, 'N' for no)"}, {'name': 'ratecodeid', 'type': 'bigint', 'purpose': 'The rate code applied to the trip (1=Standard rate, 2=JFK, 3=Newark, 4=Nassau or Westchester, 5=Negotiated fare, 6=Group ride)'}, {'name': 'pulocationid', 'type': 'int', 'purpose': 'TLC Taxi Zone ID where the passenger was picked up'}, {'name': 'dolocationid', 'type': 'int', 'purpose': 'TLC Taxi Zone ID where the passenger was dropped off'}, {'name': 'passenger_count', 'type': 'bigint', 'purpose': 'Number of passengers in the vehicle during the trip'}, {'name': 'trip_distance', 'type': 'double', 'purpose': 'Distance of the trip in miles'}, {'name': 'fare_amount', 'type': 'double', 'purpose': 'Base fare amount in dollars'}, {'name': 'extra', 'type': 'double', 'purpose': 'Extra charges (typically includes rush hour and overnight surcharges)'}, {'name': 'mta_tax', 'type': 'double', 'purpose': 'Metropolitan Transportation Authority tax amount'}, {'name': 'tip_amount', 'type': 'double', 'purpose': 'Tip amount paid by the passenger'}, {'name': 'tolls_amount', 'type': 'double', 'purpose': 'Total amount of all tolls paid during the trip'}, {'name': 'ehail_fee', 'type': 'double', 'purpose': 'E-hail fee for electronically hailed trips'}, {'name': 'improvement_surcharge', 'type': 'double', 'purpose': 'Improvement surcharge assessed on trips'}, {'name': 'total_amount', 'type': 'double', 'purpose': 'Total amount charged to the passenger (excluding cash tips)'}, {'name': 'payment_type', 'type': 'bigint', 'purpose': 'Method of payment (1=Credit card, 2=Cash, 3=No charge, 4=Dispute, 5=Unknown, 0=Other)'}, {'name': 'trip_type', 'type': 'bigint', 'purpose': 'Trip type indicator (1=Street-hail, 2=Dispatch)'}, {'name': 'congestion_surcharge', 'type': 'double', 'purpose': 'Congestion surcharge for trips in congested areas'}, {'name': 'cbd_congestion_fee', 'type': 'double', 'purpose': 'Central Business District congestion fee'}, {'name': 'tpep_pickup_datetime', 'type': 'timestamp', 'purpose': 'Date and time when the passenger was picked up (used for yellow taxis)'}, {'name': 'tpep_dropoff_datetime', 'type': 'timestamp', 'purpose': 'Date and time when the passenger was dropped off (used for yellow taxis)'}, {'name': 'airport_fee', 'type': 'double', 'purpose': 'Fee charged for airport pickups/dropoffs'}, {'name': 'taxi_class', 'type': 'string', 'purpose': 'Type of taxi (yellow or green)'}, {'name': 'year', 'type': 'string', 'purpose': 'Year when the trip took place'}, {'name': 'month', 'type': 'string', 'purpose': 'Month when the trip took place'}]}]}

## Query Execution Guidelines
1. Analyze the natural language query to understand intent and required data
2. Generate optimized Amazon Athena-compatible SQL with proper partitioning and filtering
3. For queries expected to run >30 seconds, implement 5-second sleep and status checks

## Amazon Athena Best Practices
Query Optimization:
• Use CTAS (CREATE TABLE AS SELECT) for efficient data transformation operations.
• Generate standard SQL compatible with Amazon Athena (Presto/Trino based).
• Use appropriate table joins and window functions for complex queries.
• Implement proper WHERE clause filtering for partition pruning.

## Optimize Input Tokens
1. Do not generate charts unless explicitly requested by the user.
2. If user has requested to generate charts, write code once to display charts. Do not iterate.

## Final Response:
Return the final responses strictly in a structured dictionary format:
sql_statement_list: Please print one or more SQL queries 
sql_reasoning_list: For each SQL statement generated, please explain step by step how you translated the natural language question to SQL statement.
final_response: print the answer to the question.

In [11]:
model_list = ['deepseek.v3-v1:0', 'qwen.qwen3-coder-30b-a3b-v1:0', 'us.anthropic.claude-3-7-sonnet-20250219-v1:0', 'us.anthropic.claude-sonnet-4-20250514-v1:0', 'openai.gpt-oss-20b-1:0', 'openai.gpt-oss-120b-1:0']

# We will use the following model in Strands Agent
model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0"

# Let's create a reusable function to process a query and return a response in a structured dictionary format
def  get_query_response(query, model_id="us.anthropic.claude-sonnet-4-20250514-v1:0"):

    # We want the response to be in structured dictionary format that returns SQL statement, its reasoning and the final response.
    class SQLQuery(BaseModel):
        sql_statement: str = Field(description="The SQL query that was generated")
        reasoning: str = Field(description="Step by step explanation of how the natural language question was translated to this SQL statement")

    class QueryResponse(BaseModel):        
        sql_queries: List[SQLQuery] = Field(description="List of SQL queries with their reasoning")
        final_response: str = Field(description="The final response generated")

    # Cerate the Bedrock Model using model_id
    model = BedrockModel(model_id=model_id)

    #Let's us ethe MCP client we created earlier
    with data_mcp_client:
        # Get the data processing tools from MCP server
        data_tools = data_mcp_client.list_tools_sync()

        # Optimize tools by passing just what we need instead of all 32 tools
        curated_data_tools = ['manage_aws_athena_query_executions']

        # Extract just the tools that we need.
        filtered_tools = [tool for tool in data_tools if tool.tool_name in curated_data_tools]

        #Add the following tools so we can generate charts or read / write to files if needed.
        final_tools = [python_repl, file_read, file_write] + filtered_tools
    
        # Pass the system prompt, the LLM we use with bedrock, and all the tools to the agent
        data_lake_agent = Agent(system_prompt = query_system_prompt, model=model, tools=final_tools)

        # Invoke the Agent
        temp_response = data_lake_agent(query)

        #Convert the agents response into a structured output
        response = data_lake_agent.structured_output(QueryResponse, "Extract the structured output of sql queries, reasoning, and the final response")

        #Convert the object into a dictionary
        response_dict = response.model_dump()
        return response_dict

In [12]:
response = get_query_response(f"How many rides went to Airport each month in 2025?")
pprint(response)

I'll help you find the number of rides that went to airports each month in 2025. Let me analyze the data structure and create an appropriate query.
Tool #1: manage_aws_athena_query_executions
Let me try again with a result configuration:
Tool #2: manage_aws_athena_query_executions

Tool #3: python_repl


╭───────────────────────────────────────────── Executing Python Code ─────────────────────────────────────────────╮
│ import time                                                                                                     │
│ time.sleep(5)                                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Running in standard mode...

✓ Code executed successfully (5.01s)
User objects in namespace: 14 items
 - bucket_name = '000384376621-us-west-2-big-data-agent-workshop'
 - download_dir = 'download'
 - url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-06.parquet'
 - file_path = 'download/green_tripdata_2025-06.parquet'
 - filename = 'green_tripdata_2025-06.parquet'
 - taxi_type = 'green'
 - year = '2025'
 - month = '06'
 - s3_key = 'data/taxi_class=green/year=2025/month=06/green_tripdata_2025-06.parquet'
 - s3_uri = 
's3://000384376621-us-west-2-big-data-agent-workshop/data/taxi_class=green/year=2025/month=06/green_tripdata_2025-0
6.parquet'
 - policy_str = 
'{"Version":"2012-10-17","Statement":[{"Sid":"DenyInsecureConnections","Effect":"Deny","Principal":"*","Action":"s3
:*","Resource":["arn:aws:s3:::000384376621-us-west-2-big-data-agent-workshop","arn:aws:s3:::000384376621-us-west-2-
big-data-agent-workshop/*"],"Condition":{"Bool":{"aws:SecureTransport":"false"}}},{"Sid":"DenyPublicReadWrite","Eff
ect":"Deny","Principal":"*","Action":["s3:GetObject","s3:PutObject","s3:DeleteObject","s3:GetObjectAcl","s3:PutObje
ctAcl","s3:ListBucket"],"Resource":["arn:aws:s3:::000384376621-us-west-2-big-data-agent-workshop","arn:aws:s3:::000
384376621-us-west-2-big-data-agent-workshop/*"],"Condition":{"StringNotLike":{"aws:PrincipalArn":["arn:aws:iam::000
384376621:role/AwsLabBigDataAgentStack-BigDataAgentRole-2oCLXxnlPxXX","arn:aws:iam::000384376621:root"]},"StringNot
Equals":{"aws:PrincipalServiceName":["glue.amazonaws.com","athena.amazonaws.com"]}}},{"Sid":"AllowAuthorizedAccess"
,"Effect":"Allow","Principal":{"AWS":["arn:aws:iam::000384376621:role/AwsLabBigDataAgentStack-BigDataAgentRole-2oCL
XxnlPxXX","arn:aws:iam::000384376621:root"]},"Action":["s3:GetObject","s3:PutObject","s3:DeleteObject","s3:ListBuck
et","s3:GetBucketLocation","s3:GetObjectLegalHold","s3:PutObjectLegalHold","s3:GetObjectRetention","s3:PutObjectRet
ention","s3:BypassGovernanceRetention"],"Resource":["arn:aws:s3:::000384376621-us-west-2-big-data-agent-workshop","
arn:aws:s3:::000384376621-us-west-2-big-data-agent-workshop/*"]},{"Sid":"AllowAWSServices","Effect":"Allow","Princi
pal":{"Service":["glue.amazonaws.com","athena.amazonaws.com"]},"Action":["s3:GetObject","s3:PutObject","s3:ListBuck
et","s3:GetBucketLocation"],"Resource":["arn:aws:s3:::000384376621-us-west-2-big-data-agent-workshop","arn:aws:s3::
:000384376621-us-west-2-big-data-agent-workshop/*"]}]}'
 - role = 'arn:aws:iam::000384376621:root'
 - service = 'athena.amazonaws.com'
 - local_path = 'download/green_tripdata_2025-06.parquet'


Tool #4: manage_aws_athena_query_executions
Great! The query completed successfully. Now let me get the results:
Tool #5: manage_aws_athena_query_executions
Perfect! I found the data for airport rides in 2025. Let me format this information properly:

## Response

**sql_statement_list:**
```sql
SELECT 
    month,
    COUNT(*) as airport_rides
FROM awslabbigdataagentstack_db.data
WHERE year = '2025'
    AND (airport_fee > 0 OR ratecodeid = 2 OR ratecodeid = 3)
GROUP BY month
ORDER BY CAST(month AS INTEGER)
```

**sql_reasoning_list:**
1. **Table Selection**: I queried the `awslabbigdataagentstack_db.data` table which contains NYC taxi trip records
2. **Airport Identification**: I identified airport rides using three criteria:
   - `airport_fee > 0`: Direct indicator of airport-related fees
   - `ratecodeid = 2`: JFK airport rate code
   - `ratecodeid = 3`: Newark airport rate code
3. **Year Filtering**: Applied `WHERE year = '2025'` to get only 2025 data, leveraging the year partition 

In [ ]:
response = get_query_response(f"How many taxi vendors are there? Plot a bar chart with ride count and fare amount.")
print(response)

In [ ]:
response = get_query_response("For the top 25 percentile of ride fares per yellow / green class, what is ratio of tips to total fare? If I am a taxi driver which routes and times should I drive to get the most tips?")
pprint(response)